<a href="https://colab.research.google.com/github/semishen/ML100Days/blob/master/Day_026_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成計數編碼以及搭配邏輯斯迴歸的預測 (In[4], Out[4], In[5], Out[5]) 
- 仿造範例, 完成雜湊編碼, 以及計數編碼+雜湊編碼 搭配邏輯斯迴歸的預測 (In[6], Out[6], In[7], Out[7]) 
- 試著回答上述執行結果的觀察

# 作業1
* 參考範例，將鐵達尼的艙位代碼( 'Cabin' )欄位使用特徵雜湊 / 標籤編碼 / 計數編碼三種轉換後，  
與其他類別型欄位一起預估生存機率

In [44]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

df_train = pd.read_csv('titanic_train.csv')
df_test = pd.read_csv('titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [51]:
#只取類別值 (object) 型欄位, 存於 object_features 中
# object_features = []
# for dtype, feature in zip(df.dtypes, df.columns):
#     if dtype == 'object':
#         object_features.append(feature)
# print(f'{len(object_features)} Object Features : {object_features}\n')
obj_list = df.columns[df.dtypes == 'object']
df_obj = df.loc[:,df.dtypes == 'object']
df_obj = df_obj.fillna('None')
df_obj.head()

# # 只留類別型欄位
# df = df[object_features]
# df = df.fillna('None')
# train_num = train_Y.shape[0]
# df.head()

,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
### Q: 承上題，三者比較效果何者最好?
### A: 三者比較起來 'Cabin'特徵雜湊 + 邏輯斯迴歸所得分數 (0.7822672776348002) 略高於計數編碼 + 邏輯斯迴歸，但兩者差異很小；不過 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸所得到的分數 (0.7732659594501287) 卻低於對照組 (0.7800138095537004)。



In [5]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸 0.7800138095537004
train_num = train_Y.shape[0]
df_temp = pd.DataFrame()
for c in df_obj.columns:
    df_temp[c] = LabelEncoder().fit_transform(df_obj[c])
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.7800138095537004


,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


In [16]:

col_Cabin = list(df_obj['Cabin'].value_counts().index)
counting_Cabin = list(df_obj['Cabin'].value_counts().values)
d = {'Cabin': col_Cabin, 'Cabin_Counting': counting_Cabin}
df_Cabin_Counting = pd.DataFrame(data=d)
df_obj = pd.merge(df_obj, df_Cabin_Counting, on='Cabin', how='left')
df_obj

,Name,Sex,Ticket,Cabin,Embarked,Cabin_Counting
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S,1014
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C,2
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S,1014
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S,2
4,"Allen, Mr. William Henry",male,373450,None,S,1014
...,...,...,...,...,...,...
1304,"Spector, Mr. Woolf",male,A.5. 3236,None,S,1014
1305,"Oliva y Ocana, Dona. Fermina",female,PC 17758,C105,C,1
1306,"Saether, Mr. Simon Sivertsen",male,SOTON/O.Q. 3101262,None,S,1014
1307,"Ware, Mr. Frederick",male,359309,None,S,1014


In [52]:
# 加上 'Cabin' 欄位的計數編碼
df_Cabin_Counting = df_obj.groupby(['Cabin']).size().reset_index()
df_Cabin_Counting.columns = ['Cabin','Cabin_Counting'] 
df_obj = pd.merge(df_obj, df_Cabin_Counting, on=['Cabin'], how='left')
df_obj

,Name,Sex,Ticket,Cabin,Embarked,Cabin_Counting
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S,1014
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C,2
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S,1014
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S,2
4,"Allen, Mr. William Henry",male,373450,None,S,1014
...,...,...,...,...,...,...
1304,"Spector, Mr. Woolf",male,A.5. 3236,None,S,1014
1305,"Oliva y Ocana, Dona. Fermina",female,PC 17758,C105,C,1
1306,"Saether, Mr. Simon Sivertsen",male,SOTON/O.Q. 3101262,None,S,1014
1307,"Ware, Mr. Frederick",male,359309,None,S,1014


In [58]:
# 'Cabin'計數編碼 + 邏輯斯迴歸 0.7822547234950725
train_num = train_Y.shape[0]
df_tmp = pd.DataFrame()
for c in obj_list:
  df_tmp[c] = LabelEncoder().fit_transform(df_obj[c])
df_tmp['Cabin_Counting'] = df_obj['Cabin_Counting']

train_X = df_tmp[:train_num]
estimator = LogisticRegression()
print('shape: ', train_X.shape)
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())

df_tmp.head()

shape:  (891, 6)
0.7822547234950725


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Counting
0,155,1,720,185,3,1014
1,286,0,816,106,0,2
2,523,0,914,185,3,1014
3,422,0,65,70,3,2
4,22,1,649,185,3,1014


In [59]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸 0.7822672776348002
df_tmp = pd.DataFrame()
for c in obj_list:
    df_tmp[c] = LabelEncoder().fit_transform(df_obj[c])

df_tmp['Cabin_Hash'] = df_obj['Cabin'].map(lambda x:hash(x) % 10)
train_X = df_tmp[:train_num]
estimator = LogisticRegression()
print('shape: ', train_X.shape)
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_tmp.head()

shape:  (891, 6)
0.7822672776348002


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Hash
0,155,1,720,185,3,2
1,286,0,816,106,0,0
2,523,0,914,185,3,2
3,422,0,65,70,3,3
4,22,1,649,185,3,2


In [60]:
# 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸 0.7732659594501287
df_tmp = pd.DataFrame()
for c in obj_list:
    df_tmp[c] = LabelEncoder().fit_transform(df_obj[c])

df_tmp['Cabin_Hash'] = df_obj['Cabin'].map(lambda x:hash(x) % 10)
df_tmp['Cabin_Counting'] = df_obj['Cabin_Counting']
train_X = df_tmp[:train_num]
estimator = LogisticRegression()
print('shape: ', train_X.shape)
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_tmp.head()

shape:  (891, 7)
0.7732659594501287


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Hash,Cabin_Counting
0,155,1,720,185,3,2,1014
1,286,0,816,106,0,0,2
2,523,0,914,185,3,2,1014
3,422,0,65,70,3,3,2
4,22,1,649,185,3,2,1014
